In [70]:
###Dependecies
import sqlite3
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [4]:
steam_games_df = pd.read_csv("db_files/steam_games.csv", encoding = 'utf-8', dtype={'name':str})
steam_games_df.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


In [260]:
#remove uneeded rows
#delete_labels = ['url', 'types', 'desc_snippet', 'popular_tags', 'game_details', 'languages', 'achievements',
#                 'game_description', 'mature_content', 'minimum_requirements', 'recommended_requirements', 'discount_price']
#steam_games_df = steam_games_df.drop(columns=delete_labels)
steam_games_df.rename(columns = {'name': 'Game Title',
                                 'recent_reviews': 'Recent Reviews',
                                 'all_reviews': 'All Reviews',
                                 'release_date': 'Release Date',
                                 'developer': 'Developer',
                                 'publisher': 'Publisher',
                                 'genre': 'Genre',
                                 'original_price': 'Original Price'
                                }, inplace = True)
steam_games_df.head()

,Game Title,Recent Reviews,All Reviews,Release Date,Developer,Publisher,Genre,Original Price
0,DOOM,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks",Action,$19.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Action,Adventure,Massively Multiplayer",$29.99
2,BATTLETECH,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Action,Adventure,Strategy",$39.99
3,DayZ,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Action,Adventure,Massively Multiplayer",$44.99
4,EVE Online,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Action,Free to Play,Massively Multiplayer,RPG,...",Free


In [255]:
steam_games_df['name'] = steam_games_df['name'].astype('str')
steam_games_df['name'].str.encode('ascii', 'ignore').str.decode('ascii')

#steam_games_df.loc[steam_games_df['name'] == game_names]
print(len(game_names))
temp_list = steam_games_df['name'].map(lambda x: 1 if x in game_names else 0)
temp_list.value_counts()

236


0    40661
1      172
Name: name, dtype: int64

In [268]:
steam_games_sorted_df = pd.DataFrame()
steam_games_sorted_df = steam_games_df
#steam_games_sorted_df = steam_games_df.set_index('Game Title')
steam_games_sorted_df
#for i in range(len(game_names)):
    #steam_games_sorted_df.index[i]

,Game Title,Recent Reviews,All Reviews,Release Date,Developer,Publisher,Genre,Original Price
0,DOOM,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks",Action,$19.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Action,Adventure,Massively Multiplayer",$29.99
2,BATTLETECH,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Action,Adventure,Strategy",$39.99
3,DayZ,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Action,Adventure,Massively Multiplayer",$44.99
4,EVE Online,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Action,Free to Play,Massively Multiplayer,RPG,...",Free
...,...,...,...,...,...,...,...,...
40828,Rocksmith® 2014 Edition – Remastered – Sabaton...,NaN,NaN,"Feb 12, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation",$2.99
40829,Rocksmith® 2014 Edition – Remastered – Stone T...,NaN,NaN,"Feb 5, 2019",Ubisoft - San Francisco,NaN,"Casual,Simulation",$2.99
40830,Fantasy Grounds - Quests of Doom 4: A Midnight...,NaN,NaN,"Jul 31, 2018","SmiteWorks USA, LLC",NaN,"Indie,RPG,Strategy",$7.99
40831,Mega Man X5 Sound Collection,NaN,NaN,"Jul 24, 2018","CAPCOM CO., LTD","CAPCOM CO., LTD,CAPCOM CO., LTD",Action,$9.99


In [273]:
steam_games_sorted_df.dtypes

Game Title        object
Recent Reviews    object
All Reviews       object
Release Date      object
Developer         object
Publisher         object
Genre             object
Original Price    object
dtype: object

In [272]:
final_steam_df = pd.DataFrame()
final_steam_df = kickstarter_df.merge(steam_games_sorted_df, how='inner')
final_steam_df

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [265]:
#steam_games_sorted_df.loc[game_names]
#steam_games_sorted_df[steam_games_sorted_df.index.duplicated()]
#steam_games_sorted_df.reindex(game_names, fill_value = 'Missing')

In [7]:
metacritic_df = pd.read_csv("db_files/metacritic_games.csv", encoding = 'utf-8')
metacritic_df

,game,platform,developer,genre,number_players,rating,release_date,positive_critics,neutral_critics,negative_critics,positive_users,neutral_users,negative_users,metascore,user_score
0,Portal 2,PC,Valve Software,Action,NaN,E10+,"Apr 18, 2011",51,1,0,1700,107,19,95,90
1,The Elder Scrolls V: Skyrim,PC,Bethesda Game Studios,Role-Playing,No Online Multiplayer,M,"Nov 10, 2011",32,0,0,1616,322,451,94,82
2,The Legend of Zelda: Ocarina of Time 3D,3DS,GREZZO,Miscellaneous,No Online Multiplayer,E10+,"Jun 19, 2011",84,1,0,283,20,5,94,90
3,Batman: Arkham City,PC,Rocksteady Studios,Action Adventure,NaN,T,"Nov 21, 2011",27,0,0,240,34,27,91,87
4,Super Mario 3D Land,3DS,Nintendo,Action,No Online Multiplayer,E,"Nov 13, 2011",81,1,0,251,39,11,90,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5694,Modern Combat: Blackout,Switch,Gameloft,Action,No Online Multiplayer,NaN,"Feb 14, 2019",0,5,2,1,0,1,53,57
5695,Fimbul,PS4,Zaxis,Action Adventure,No Online Multiplayer,NaN,"Feb 28, 2019",0,7,2,1,0,0,52,18
5696,Generation Zero,PC,Avalanche Studios,Action Adventure,Online Multiplayer,T,"Mar 26, 2019",1,11,6,5,2,6,50,57
5697,Vane,PS4,Friend & Foe,Adventure,No Online Multiplayer,E10+,"Jan 15, 2019",3,26,22,2,2,2,49,55


In [191]:
metacritic_df.loc[metacritic_df['platform']=='PC']

,game,platform,developer,genre,number_players,rating,release_date,positive_critics,neutral_critics,negative_critics,positive_users,neutral_users,negative_users,metascore,user_score
0,Portal 2,PC,Valve Software,Action,NaN,E10+,"Apr 18, 2011",51,1,0,1700,107,19,95,90
1,The Elder Scrolls V: Skyrim,PC,Bethesda Game Studios,Role-Playing,No Online Multiplayer,M,"Nov 10, 2011",32,0,0,1616,322,451,94,82
3,Batman: Arkham City,PC,Rocksteady Studios,Action Adventure,NaN,T,"Nov 21, 2011",27,0,0,240,34,27,91,87
5,Deus Ex: Human Revolution,PC,Nixxes Software,Action,No Online Multiplayer,M,"Aug 23, 2011",52,0,0,520,112,78,90,85
7,Total War: Shogun 2,PC,Creative Assembly,Strategy,Online Multiplayer,T,"Mar 15, 2011",62,0,0,187,44,29,90,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5670,Jon Shafer's At the Gates,PC,Conifer Games,Strategy,NaN,NaN,"Jan 23, 2019",4,16,3,1,0,0,65,63
5685,Jump Force,PC,"Spike Chunsoft Co. Ltd., Spike Chunsoft",Action Adventure,Online Multiplayer,T,"Feb 15, 2019",2,6,1,1,4,17,60,41
5688,Anthem,PC,BioWare,Role-Playing,Up to 4,T,"Feb 22, 2019",5,59,7,446,117,616,59,42
5693,Crackdown 3,PC,"Sumo Digital, Elbow Rocket",Action Adventure,Up to 10,M,"Feb 15, 2019",2,11,3,12,4,15,55,29


In [8]:
kickstarter_df = pd.read_csv("db_files/ks-projects-201801.csv", encoding = 'utf-8')
kickstarter_df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [9]:
#considering taking this out and doing more web scraping to get relevant values

#sort kickstarter df for just video games
ks_games_df = kickstarter_df.loc[kickstarter_df['category']=='Video Games']
#sort for successful ks campaigns
ks_games_df = ks_games_df.loc[ks_games_df['state'] == 'successful']
ks_games_df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
126,1000524949,Operation: Make Stuff,Video Games,Games,USD,2012-10-18,200.0,2012-09-18 04:46:06,306.72,successful,36,US,306.72,306.72,200.00
199,1000811882,The Sword of Asumi Visual Novel,Video Games,Games,GBP,2014-11-10,2500.0,2014-10-19 20:35:23,3097.00,successful,107,GB,4983.78,4848.23,3913.65
219,1000940032,Space Trucker,Video Games,Games,USD,2016-12-08,150.0,2016-11-08 08:20:27,165.00,successful,7,US,25.00,165.00,150.00
319,1001442379,The Contractors - Create 3D Model: Prop Asset ...,Video Games,Games,USD,2017-05-26,2000.0,2017-05-05 12:47:05,2348.00,successful,17,US,340.00,2348.00,2000.00
475,1002272545,Course: Make a Stunning Fantasy RPG with GameM...,Video Games,Games,USD,2016-08-13,10200.0,2016-07-14 00:01:11,20709.50,successful,1258,US,11973.50,20709.50,10200.00


In [197]:
ign_df = pd.read_csv("db_files/IGN_reviews.csv", encoding = 'utf-8')
ign_df_pc = ign_df.loc[ign_df['Platform']=='PC']
ign_df_pc

,Game,Platform,Score,Genre
4,Sportsfriends,PC,8.7,"Action, Compilation"
7,The Walking Dead: Season Two -- Episode 3: In ...,PC,9.0,Adventure
9,Bound by Flame,PC,7.0,"Action, RPG"
12,Outlast: Whistleblower,PC,6.4,"Action, Adventure"
14,Tesla Effect: A Tex Murphy Adventure,PC,6.8,Adventure
...,...,...,...,...
17088,Army Men,PC,4.0,Action
17089,Dominion: Storm Over Gift 3,PC,4.0,Strategy
17090,Revenge of Arcade,PC,6.0,Action
17091,Addiction Pinball,PC,8.0,Pinball


In [219]:
ign_df_sorted = pd.DataFrame()
for i in range(len(game_names)):
    try:
        ign_df_sorted = ign_df.loc[ign_df['Game'] == game_names[i]]
    except:
        print(f'game not found: {game_names[i]} index: ' + i)
    
ign_df_sorted

,Game,Platform,Score,Genre


In [222]:
ign_df.loc[ign_df['Game']== "Octodad: Dadliest Catch"]

,Game,Platform,Score,Genre
125,Octodad: Dadliest Catch,PC,7.8,Adventure


In [211]:
for i in range(len(game_names)):
    print(game_names[i])

7 Days to Die
A Wizard's Lizard
Adventurezator: When Pigs Fly
Akaneiro: Demon Hunters
All Walls Must Fall - A Tech-Noir Tactics Game
Among the Sleep - Enhanced Edition
Anima Gate of Memories
AR-K
Armello
Armikrog
AVARIAvs
Awesomenauts - the 2D moba
Battle Worlds: Kronos
Bear Simulator
Black The Fall
Blade Symphony
Bloodstained: Ritual of the Night
BOMBFEST
Boss Monster
Broken Age
Broken Sword 5 - the Serpent's Curse
C-Wars
Castle Story
Catlateral Damage
Caveman Warriors
Chaos Reborn
Chasm
Children of Liberty
Children of Morta
Children of Zodiarcs
Chivalry: Medieval Warfare
Chroma Squad
Chuck's Challenge 3D 2020
CLANNAD
Cognition: An Erica Reed Thriller
Conclave
CONSORTIUM
Contagion
Cosmic Star Heroine
Crea
CreaVures
Darkest Dungeon®
Dead State: Reanimated
Desert Child
Dex
Die for Valhalla!
Rogue Stormers
Divinity: Original Sin (Classic)
Dragon Fin Soup
Drifter
DubWars
Duelyst
Dungeon of Elements
Dungeonmans
Dyscourse
Dysfunctional Systems: Learning to Manage Chaos
Edge of Space
Elite D

In [128]:
#Now to scrape the kickstarter page for successful games that launched on steam
#looking for the names of each game and potentially other info that might be interesting to add to the db
url='https://www.kickstarter.com/play?sort=alpha#all-projects'
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\pwitw\.wdm\drivers\chromedriver\win32\87.0.4280.88]


In [153]:
#set initial url and retrieve html for the first page
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#list for storing the links to the games' corresponding steam pages
steam_links=[]
counter = 0

#while loop executes as long as the new page contains a "view more" button
#This means it will not collect links from the final page, which doesn't have the button
while soup.find('a', class_='more btn btn--border-gray', href=True) is not None:
    
    #inital .find for the html elements
    ks_pages = soup.find('section', {'id' : 'all-projects'})
    
    #find the class with the steam link and save it
    for a in ks_pages.find_all('a', class_= 'hover-zoomout f3 bg-green-90 track-service', href=True):
        try:
            steam_links.append(a['href'])
        except KeyError:
            pass
    #check for more pages, browser.visit takes us to the next page if it's there
    try:
        button = soup.find('a', class_='more btn btn--border-gray', href=True)['href']
        browser.visit(button)
        counter += 1
        print(f"clicks: {counter}")
        time.sleep(1)
    except TypeError:
        pass
    
    #reset html and soup for the new page
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
#for the final page that does not have the view more button
ks_pages = soup.find('section', {'id' : 'all-projects'})
    
#find the class with the steam link and save it
for a in ks_pages.find_all('a', class_= 'hover-zoomout f3 bg-green-90 track-service', href=True):
    try:
        steam_links.append(a['href'])
    except KeyError:
        pass
    
#check that we retrieved all the links
len(steam_links)

clicks: 1
clicks: 2
clicks: 3
clicks: 4
clicks: 5
clicks: 6
clicks: 7
clicks: 8
clicks: 9
clicks: 10
clicks: 11
clicks: 12
clicks: 13
clicks: 14
clicks: 15
clicks: 16


255

In [164]:
#now to scrape steam for the name of the game
length = len(steam_links)
game_names = []
for i in range(length):
    browser.visit(steam_links[i])
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        game_names.append(soup.find(class_='apphub_AppName').text)
    except:
        print(f"unexpected error: {steam_links[i]} index: {i}")
    time.sleep(1)

unexpected error: https://store.steampowered.com/app/249380 index: 22
unexpected error: https://store.steampowered.com/app/210870 index: 35
unexpected error: https://store.steampowered.com/app/258030 index: 58
unexpected error: https://store.steampowered.com/app/299700 index: 64
unexpected error: https://store.steampowered.com/app/201040 index: 74
unexpected error: https://store.steampowered.com/app/379430 index: 104
unexpected error: https://store.steampowered.com/app/234980 index: 110
unexpected error: https://store.steampowered.com/app/280440 index: 121
unexpected error: https://store.steampowered.com/app/218820 index: 127
unexpected error: https://store.steampowered.com/app/449830 index: 133
unexpected error: https://store.steampowered.com/app/246400 index: 134
unexpected error: https://store.steampowered.com/app/233250 index: 160
unexpected error: https://store.steampowered.com/app/96300 index: 165
unexpected error: https://store.steampowered.com/app/266550 index: 194
unexpected e

In [ ]:
# The errors I found were links to the steam homepage rather than to a specific game.
# There could be a number of reasons why these games are no longer available on steam.
# In the first case with the game 'Carmageddon: Reincarnation' (index[22]) steam still offered other titles 
# from the carmageddon series and it's possible the game is now bundled with other games or is no longer available under the 
# same title listed on kickstarter.

# In another instance, I noticed the correct page was not being pulled up because it required the user to verify their
# age due to graphic content.

In [165]:
browser.visit(steam_links[75])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

soup.find(class_='apphub_AppName').text

'Garden Paws'

In [175]:
print(game_names[22])
#len(game_names)
# the proper steam link for "Castle Story" is at steam_links[23] because steam_link[22] didn't return a game name
print(steam_links[23])

copy_steam_links = steam_links
print(len(steam_links))
print(len(copy_steam_links))
print(copy_steam_links[23])

Castle Story
https://store.steampowered.com/app/227860
255
255
https://store.steampowered.com/app/227860


In [172]:
lost_link_indexes = [22, 35, 58, 64, 74, 104, 110, 121, 127, 133, 134, 160, 165, 194, 196, 202, 221, 237, 250]
#adjust for changing the length of the steam_links list

index_adjust = 0
for i in range(len(lost_link_indexes)):
    lost_link_indexes[i] = lost_link_indexes[i]-index_adjust
    index_adjust +=1
print(lost_link_indexes)



[22, 34, 56, 61, 70, 99, 104, 114, 119, 124, 124, 149, 153, 181, 182, 187, 205, 220, 232]


In [176]:
# careful, running this code multiple times will cause problems
list_to_verify=[]
length = len(lost_link_indexes)
#for i in range(length):
#    list_to_verify.append(copy_steam_links[lost_link_indexes[i]])
#    copy_steam_links.pop(lost_link_indexes[i])

In [263]:
# Finally create dataframe containing the names of games scraped from kickstarter and their steam store urls
webscraped_df = pd.DataFrame(columns=['Game Title', 'Steam Store Url'])
webscraped_df['Game Title'] = game_names
webscraped_df['Steam Store Url'] = copy_steam_links
webscraped_df

,Game Title,Steam Store Url
0,7 Days to Die,https://store.steampowered.com/app/251570
1,A Wizard's Lizard,https://store.steampowered.com/app/280040
2,Adventurezator: When Pigs Fly,https://store.steampowered.com/app/300280
3,Akaneiro: Demon Hunters,https://store.steampowered.com/app/230310
4,All Walls Must Fall - A Tech-Noir Tactics Game,https://store.steampowered.com/app/628290
...,...,...
231,WARMACHINE: Tactics,https://store.steampowered.com/app/253510
232,Whispering Willows,https://store.steampowered.com/app/288060
233,Windforge,https://store.steampowered.com/app/266170
234,Wings! Remastered Edition,https://store.steampowered.com/app/320840


In [274]:
webscraped_df.dtypes

Game Title         object
Steam Store Url    object
dtype: object